In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import *

spark_master = "spark://e5616452f8d5:7077"

spark = SparkSession.builder \
        .appName("prueba_kafka") \
        .master(spark_master) \
        .config('spark.executor.memory', '1g') \
        .config('spark.driver.memory', '1g') \
        .config("spark.executor.cores", "1") \
        .config("spark.cores.max", "2") \
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.4.1,org.apache.kudu:kudu-spark3_2.12:1.17.0") \
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

print(spark.sparkContext.uiWebUrl)

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
org.apache.kudu#kudu-spark3_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f8fc6b9d-7c0e-464c-b898-42ed6c09d12b;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.4.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.4.1 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central


http://e5616452f8d5:4041


In [5]:
import time

# Leer las tablas Kudu en modo batch
kudu_masters = "kudu-master-1:7051,kudu-master-2:7151,kudu-master-3:7251"

# Registrar kudu_table_1 y kudu_table_2 como vistas temporales
spark.read.format("kudu") \
    .option("kudu.master", kudu_masters) \
    .option("kudu.table", "impala::s_productos.k_trades") \
    .load() \
    .createOrReplaceTempView("k_trades")

# spark.read.format("kudu") \
#     .option("kudu.master", kudu_masters) \
#     .option("kudu.table", "impala::s_productos.k_pairs_avg_prices") \
#     .load() \
#     .createOrReplaceTempView("k_pairs_avg_prices")

# # Realizar la unión y cualquier transformación usando SQL
# joined_df_sql = spark.sql("""
#     SELECT count(*) cantidad
#     FROM k_trades;
# """)

# # Escritura del resultado en una tabla de Kudu
# def write_to_kudu(batch_df, batch_id):
#     print(f"Batch ID: {batch_id}")
#     batch_df.show(truncate=False)

# # Usar el DataFrame unido en SQL para la salida en streaming
# query = joined_df_sql.writeStream \
#     .outputMode("append") \
#     .foreachBatch(write_to_kudu) \
#     .start()

# query.awaitTermination()

# Loop para ejecutar el conteo en intervalos
try:
    while True:
        # Realizar la consulta y mostrar el resultado
        joined_df_sql = spark.sql("""
            SELECT count(*) cantidad
            FROM k_trades;
        """)
        joined_df_sql.show(truncate=False)

        # Pausa entre cada iteración (ajustar según sea necesario)
        time.sleep(1)  # Espera 10 segundos antes de la siguiente iteración

except KeyboardInterrupt:
    print("Proceso detenido manualmente.")

finally:
    spark.stop()


+--------+
|cantidad|
+--------+
|406496  |
+--------+

+--------+
|cantidad|
+--------+
|406785  |
+--------+



+--------+
|cantidad|
+--------+
|406905  |
+--------+

+--------+
|cantidad|
+--------+
|407254  |
+--------+

+--------+
|cantidad|
+--------+
|407262  |
+--------+

+--------+
|cantidad|
+--------+
|407371  |
+--------+



+--------+
|cantidad|
+--------+
|407442  |
+--------+

+--------+
|cantidad|
+--------+
|407454  |
+--------+

+--------+
|cantidad|
+--------+
|407462  |
+--------+

+--------+
|cantidad|
+--------+
|407472  |
+--------+

+--------+
|cantidad|
+--------+
|407481  |
+--------+

+--------+
|cantidad|
+--------+
|407487  |
+--------+



+--------+
|cantidad|
+--------+
|407518  |
+--------+

+--------+
|cantidad|
+--------+
|407585  |
+--------+

+--------+
|cantidad|
+--------+
|407630  |
+--------+



+--------+
|cantidad|
+--------+
|407643  |
+--------+

+--------+
|cantidad|
+--------+
|407645  |
+--------+

+--------+
|cantidad|
+--------+
|407700  |
+--------+



+--------+
|cantidad|
+--------+
|407705  |
+--------+

+--------+
|cantidad|
+--------+
|407872  |
+--------+

+--------+
|cantidad|
+--------+
|407880  |
+--------+

Proceso detenido manualmente.
